In [2]:
import tensorflow as tf

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
#from sklearn.model_selection import train_test_split
#from sklearn.utils import shuffle

import collections
import random
import re
import numpy as np
import pandas as pd
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

from tqdm.auto import tqdm
tqdm.pandas()

from pathlib import Path

import Levenshtein

import csv

In [41]:
BATCH_SIZE = 64
BUFFER_SIZE = 1000
embedding_dim = 256
units = 512
# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
features_shape = 2048
attention_features_shape = 64

In [5]:
rootpath = "data/bms-molecular-translation/"

In [6]:
train = pd.read_csv(rootpath + "train_labels.csv")

In [7]:
def get_train_file_path(image_id):
    return "../input/bms-molecular-translation/train_inception/{}/{}/{}/{}.npy".format(
        image_id[0], image_id[1], image_id[2], image_id 
    )

train['file_path'] = train['image_id'].apply(get_train_file_path)

print(f'train.shape: {train.shape}')
display(train.head())

train.shape: (2424186, 3)


,image_id,InChI,file_path
0,000011a64c74,InChI=1S/C13H20OS/c1-9(2)8-15-13-6-5-10(3)7-12...,../input/bms-molecular-translation/train_incep...
1,000019cc0cd2,InChI=1S/C21H30O4/c1-12(22)25-14-6-8-20(2)13(1...,../input/bms-molecular-translation/train_incep...
2,0000252b6d2b,InChI=1S/C24H23N5O4/c1-14-13-15(7-8-17(14)28-1...,../input/bms-molecular-translation/train_incep...
3,000026b49b7e,InChI=1S/C17H24N2O4S/c1-12(20)18-13(14-7-6-10-...,../input/bms-molecular-translation/train_incep...
4,000026fc6c36,InChI=1S/C10H19N3O2S/c1-15-10(14)12-8-4-6-13(7...,../input/bms-molecular-translation/train_incep...


In [8]:
def preprocess_label(InChI):
    return "<{}>".format(InChI.replace("InChI=1S/", ""))

In [9]:
train["InChI_clean"] = train["InChI"].apply(preprocess_label)

In [11]:
train.head()

,image_id,InChI,file_path,InChI_clean
0,000011a64c74,InChI=1S/C13H20OS/c1-9(2)8-15-13-6-5-10(3)7-12...,../input/bms-molecular-translation/train_incep...,<C13H20OS/c1-9(2)8-15-13-6-5-10(3)7-12(13)11(4...
1,000019cc0cd2,InChI=1S/C21H30O4/c1-12(22)25-14-6-8-20(2)13(1...,../input/bms-molecular-translation/train_incep...,<C21H30O4/c1-12(22)25-14-6-8-20(2)13(10-14)11-...
2,0000252b6d2b,InChI=1S/C24H23N5O4/c1-14-13-15(7-8-17(14)28-1...,../input/bms-molecular-translation/train_incep...,<C24H23N5O4/c1-14-13-15(7-8-17(14)28-12-10-20(...
3,000026b49b7e,InChI=1S/C17H24N2O4S/c1-12(20)18-13(14-7-6-10-...,../input/bms-molecular-translation/train_incep...,<C17H24N2O4S/c1-12(20)18-13(14-7-6-10-24-14)11...
4,000026fc6c36,InChI=1S/C10H19N3O2S/c1-15-10(14)12-8-4-6-13(7...,../input/bms-molecular-translation/train_incep...,<C10H19N3O2S/c1-15-10(14)12-8-4-6-13(7-8)5-2-3...


In [12]:
# tokenizing
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token="q", filters=' ', lower=False, char_level=True)
tokenizer.fit_on_texts(train["InChI_clean"].values)
tokenizer.word_index['q'] = 0
tokenizer.index_word[0] = 'q'

ValueError: Wrong number of items passed 396, placement implies 1

In [42]:
vocab_size = max(tokenizer.index_word.keys())+1

In [13]:
train_token = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(train["InChI_clean"].values), padding='post')

In [14]:
train_token

array([[19, 18,  2, ...,  0,  0,  0],
       [19, 18,  4, ...,  0,  0,  0],
       [19, 18,  4, ...,  0,  0,  0],
       ...,
       [19, 18,  4, ...,  0,  0,  0],
       [19, 18,  2, ...,  0,  0,  0],
       [19, 18,  2, ...,  0,  0,  0]], dtype=int32)

In [15]:
paths = train["file_path"]

In [18]:
image_count = len(paths)

In [19]:
list_ds = tf.data.Dataset.from_tensor_slices((paths, train_token))
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)

for f in list_ds.take(5):
    print(f)

(<tf.Tensor: shape=(), dtype=string, numpy=b'../input/bms-molecular-translation/train_inception/b/2/b/b2b3e3c06fca.png'>, <tf.Tensor: shape=(396,), dtype=int32, numpy=
array([19, 18,  2,  4, 10,  4,  2, 23,  5, 24, 17, 20,  2,  3,  5,  3,  9,
        3,  2,  4,  7,  2, 12,  3,  4,  8,  2,  2,  7,  2,  9,  8, 13,  3,
       15,  3, 14,  3,  2, 11,  3, 12,  3, 11,  3,  2, 16,  7, 15,  8,  2,
        5, 17, 22, 11,  3, 12,  6, 14,  6,  2,  2,  3,  2,  4, 10,  6,  5,
        3,  9,  6, 13,  6,  2,  9, 10,  4,  6,  2,  3,  4, 10,  5,  6,  7,
       10,  4,  6,  2,  5,  6,  2, 11,  8, 21,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       

In [20]:
val_size = int(image_count * 0.2)
train_ds = list_ds.skip(val_size)
val_ds = list_ds.take(val_size)

print(tf.data.experimental.cardinality(train_ds).numpy())
print(tf.data.experimental.cardinality(val_ds).numpy())

1939349
484837


In [44]:
num_steps = tf.data.experimental.cardinality(train_ds).numpy() // BATCH_SIZE

In [21]:
autotune = tf.data.AUTOTUNE

In [38]:
def map_func(image_path, label):
    img_tensor = np.load(image_path.decode('utf-8'))
    return img_tensor, label

In [39]:
train_ds = train_ds.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.AUTOTUNE)

In [40]:
def configure_for_performance(ds, autotune=tf.data.AUTOTUNE):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=BUFFER_SIZE)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=autotune)
    return ds

In [ ]:
train_ds = configure_for_performance(train_ds, batch_size=batch_size, autotune=autotune)
val_ds = configure_for_performance(val_ds, batch_size=1, autotune=autotune)

In [ ]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)

        # hidden shape == (batch_size, hidden_size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        # attention_hidden_layer shape == (batch_size, 64, units)
        attention_hidden_layer = (tf.nn.tanh(self.W1(features) +
                                             self.W2(hidden_with_time_axis)))

        # score shape == (batch_size, 64, 1)
        # This gives you an unnormalized score for each image feature.
        score = self.V(attention_hidden_layer)

        # attention_weights shape == (batch_size, 64, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [ ]:
class CNN_Encoder(tf.keras.Model):
    # Since you have already extracted the features and dumped it using pickle
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

In [ ]:
class RNN_Decoder(tf.keras.Model):
    def __init__(self, embedding_dim, units, vocab_size):
        super(RNN_Decoder, self).__init__()
        self.units = units

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc1 = tf.keras.layers.Dense(self.units)
        self.fc2 = tf.keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.units)
 
    def call(self, x, features, hidden):
        # defining attention as a separate model
        context_vector, attention_weights = self.attention(features, hidden)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # shape == (batch_size, max_length, hidden_size)
        x = self.fc1(output)

        # x shape == (batch_size * max_length, hidden_size)
        x = tf.reshape(x, (-1, x.shape[2]))

        # output shape == (batch_size * max_length, vocab)
        x = self.fc2(x)

        return x, state, attention_weights

    def reset_state(self, batch_size):
        return tf.zeros((batch_size, self.units))

In [ ]:
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [45]:
checkpoint_path = "./checkpoints/attention_rnn_train"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

30302

In [ ]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
    # restoring the latest checkpoint in checkpoint_path
    ckpt.restore(ckpt_manager.latest_checkpoint)

In [ ]:
# adding this in a separate cell because if you run the training cell
# many times, the loss_plot array will be reset
loss_plot = []

In [ ]:
@tf.function
def train_step(img_tensor, target):
    loss = 0

    # initializing the hidden state for each batch
    # because the captions are not related from image to image
    hidden = decoder.reset_state(batch_size=target.shape[0])

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)

    with tf.GradientTape() as tape:
        features = encoder(img_tensor)

        for i in range(1, target.shape[1]):
            # passing the features through the decoder
            predictions, hidden, _ = decoder(dec_input, features, hidden)

            loss += loss_function(target[:, i], predictions)
            
            # using teacher forcing
            dec_input = tf.expand_dims(target[:, i], 1)

    total_loss = (loss / int(target.shape[1]))

    trainable_variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, trainable_variables)

    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss, total_loss

In [ ]:
EPOCHS = 20

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(
              epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    if epoch % 5 == 0:
        ckpt_manager.save()

    print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                         total_loss/num_steps))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [ ]:
plt.plot(loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.show()

# Evaluation

In [46]:
test = pd.read_csv(rootpath + "sample_submission.csv")

In [48]:
def get_train_file_path(image_id):
    return rootpath + "test/{}/{}/{}/{}.png".format(
        image_id[0], image_id[1], image_id[2], image_id 
    )

test['file_path'] = test['image_id'].apply(get_train_file_path)

print(f'train.shape: {test.shape}')
display(test.head())

train.shape: (1616107, 3)


,image_id,InChI,file_path
0,00000d2a601c,InChI=1S/H2O/h1H2,data/bms-molecular-translation/test/0/0/0/0000...
1,00001f7fc849,InChI=1S/H2O/h1H2,data/bms-molecular-translation/test/0/0/0/0000...
2,000037687605,InChI=1S/H2O/h1H2,data/bms-molecular-translation/test/0/0/0/0000...
3,00004b6d55b6,InChI=1S/H2O/h1H2,data/bms-molecular-translation/test/0/0/0/0000...
4,00004df0fe53,InChI=1S/H2O/h1H2,data/bms-molecular-translation/test/0/0/0/0000...


In [51]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img

In [50]:
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

In [ ]:
def evaluate(img_tensor_val):

    hidden = decoder.reset_state(batch_size=1)


    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)


        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    return result

In [52]:
def get_img_id(filepath):
    return filepath.split("/")[-1].replace(".png", "")

In [ ]:
result = []

for path in test['file_path']:
    img = load_image(path)
    img_tensor_val = image_features_extract_model(img)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))
    prediction = evaluate(img_tensor_val)
    result.append({"image_id": get_img_id(path), "InChI": prediction.replace("<", "InChI=1S/").replace(">","")})


In [ ]:
test_df = pd.DataFrame(result)

In [ ]:
test_df.to_csv("results_attention_rnn.csv")